In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

rcParams['figure.figsize'] = 15,10

In [2]:
kospi = pd.read_csv('./final.csv')
snp = pd.read_csv('./SPY_2011_22.csv')

In [3]:
kospi['Date'] = pd.to_datetime(kospi['Date'])
snp['Date'] = pd.to_datetime(snp['Date'])

In [4]:
# kospi와 snp의 겹치는 날짜만 가져와 데이터 프레임으로 만들기
df = pd.merge(kospi, snp, how='inner', on='Date').rename(columns={'Close_x' : 'KOSPI200', 'Close_y' : 'SPY'})

## 상관 관계 분석 - 종가

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [6]:
X = df[['SPY']]
y = df['KOSPI200']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [8]:
scaler = MinMaxScaler()  # MinMaxScaler: 0~1 사이의 값으로 변환 -> 왜 해야 하는지? 종가 범위를 좁히기 위해서
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [10]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [11]:
from sklearn.metrics import r2_score

In [12]:
print(r2_score(y_train, train_predict))

0.6674172223957985


In [13]:
print(r2_score(y_test, test_predict))

0.7066590764414845


S&P와 Kospi 종가 간의 선형 회귀 분석 결과, 0.66~0.7 사이의 결정 계수가 나타났다.  
따라서, 일정 수준 이상의 상관성을 띄는 것으로 판단된다.  
1에 가까울 수록 상관성이 높고, 0에 가까울 수록 상관성이 낮다고 본다.  
  
  
**r2_score를 사용한 목적**
- 예측할 때 LSTM의 r2_score은 만들어진 모델이 주가를 얼마나 잘 예측했느냐?를 알기 위해
- 선형 회귀의 r2_score은 Kospi와 S&P 종가 간의 상관성이 어느 정도냐?를 알기 위해

## 상관 관계 분석 - 수익률(변동률)

In [14]:
X_pct = df[['SPY']].pct_change() * 100
y_pct = df['KOSPI200'].pct_change() * 100

In [15]:
# 수익률을 구한 것이기에 비교할 날짜가 없는 제일 첫 날은 NaN 값이기에 제거
X_pct = X_pct.iloc[1:, :]
y_pct = y_pct.iloc[1:]

In [16]:
X_pct

,SPY
1,-0.055096
2,0.519764
3,-0.195863
4,-0.196248
5,-0.125842
...,...
2860,-1.426614
2861,0.575227
2862,-0.394351
2863,-1.242787


In [17]:
y_pct

1       0.719477
2      -0.155921
3      -0.228800
4       0.323966
5      -0.587787
          ...   
2860    1.401576
2861   -1.759464
2862    0.711756
2863   -2.343699
2864   -2.016224
Name: KOSPI200, Length: 2864, dtype: float64

In [18]:
X_pct_train, X_pct_test, y_pct_train, y_pct_test = train_test_split(X_pct, y_pct, random_state=42, test_size=0.2)

이미 수익률의 범위가 충분히 좁은 상태이기에 스케일링은 진행하지 않기로!

In [19]:
model_pct = LinearRegression()

model_pct.fit(X_pct_train, y_pct_train)

LinearRegression()

In [20]:
pct_train_predict = model_pct.predict(X_pct_train)
pct_test_predict = model_pct.predict(X_pct_test)

In [21]:
print(r2_score(y_pct_train, pct_train_predict))

0.050126736296977836


In [22]:
print(r2_score(y_pct_test, pct_test_predict))

0.09802413114095865


전체 기간의 수익률(변동률) 그래프를 확인해보면 Kospi는 변동성이 비교적 컸고, S&P는 비교적 꾸준한 상승세를 보였다.  
때문에 두 증시 변동률에 대한 상관 관계 분석을 진행하였을 때 결정 계수가 낮은 것으로 판단된다.

## 결론
Kospi와 S&P의 종가 간의 상관 관계와 Kospi와 S&P의 수익률(변동률) 간의 상관 관계 분석을 확인 해보았다.  
일반적으로 Kospi가 S&P의 영향을 많이 받는다고 알려져 있는데 실제 분석 결과, 종가 간에는 어느 정도 상관성을 보였으나 변동률 간에는 상관성이 거의 없는 것으로 확인되었다.  
따라서 S&P 종가 외에 원-달러 환율, 금값, 한국 금리가 Kospi에게 얼마나 많은 영향을 미치는 지 실증하기로 한다. = 다중 선형 회귀 진행하겠다.  
  
  
**다중 선형 회귀**  
y: Kospi 200  
X: 환율, 한국 금리, 금값, GDP, S&P 500 등등 (제거 또는 추가 가능)